# Data Preparation for XGBoost

This notebook collects some code for preparing a dataset for training in XGBoost.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

## Load Data

In this example we'll work on the 
[Kagle Bluebook for Bulldozers](https://www.kaggle.com/competitions/bluebook-for-bulldozers/overview)
competition, which asks us to build a regression model to predict the sale price of heavy equipment.

In [ ]:
df = pd.read_csv('Train.csv', parse_dates=['saledate']);
df.info()

## Prepare raw data for XGBoost

All feature data must be numeric, and if the target is discrete, it must be encoded in 0,1,...K.

### Encode string features

The simplest way to encode string variables is to map each unique string value to an integer;
this is called *integer encoding*.

We can easily accomplish this by using the [categorical data type in pandas](https://pandas.pydata.org/docs/user_guide/categorical.html).
The category type is a bit like the factor type in R;
pandas stores the underlying data as integers, and it keeps a mapping from the integers back to the original string values.
XGBoost is able to access the numeric data underlying the categorical features for model training and prediction.
This is a nice way to encode string features because it's easy to implement and it preserves the original category levels in the data frame.
If you prefer to generate your own integer mappings, you can also do it with the scikit-learn
[OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html).

In [ ]:
def str_cols_to_cat_cols(df, cols, as_codes=False):
    out_df = df.copy()
    for col in cols:
        if as_codes:
            out_df[col] = df[col].astype('category').cat.codes
        else:
            out_df[col] = df[col].astype('category')
    return out_df

In [ ]:
# get a list of the string columns to transform
raw_string_cols = df.dtypes[df.dtypes == 'object'].index.to_list()

In [ ]:
proc_df = str_cols_to_cat_cols(df, raw_string_cols, as_codes=False)

## Encode datetime features

While dates feel sort of numeric, they are not quite numbers, so we need to transform them  into numeric columns that XGBoosst can understand.
A single date has many different attributes, e.g. days since epoch, year, quarter, month, day, day of year, day of week, is holiday, which can be extracted.

In [ ]:
def extract_datetime_features(df, datetime_columns, include_epoch=False, datetime_attributes=None):
    out_df = df.copy()
    datetime_attributes = datetime_attributes if datetime_attributes else []
    epoch = pd.Timestamp(year=1970, month=1, day=1)
    for col in datetime_columns:
        if include_epoch:
            out_df[f'{col}_days_since_epoch'] = (out_df[col] - epoch).dt.days
        for datetime_attribute in datetime_attributes:
            out_df[f'{col}_{datetime_attribute}'] = getattr(out_df[col].dt, datetime_attribute)
    return out_df

datetime_attributes = [
    'year',
    'month',
    'day',
    'quarter',
    'day_of_year',
    'day_of_week',
]

In [ ]:
# get a list of the datetime columns
datetime_cols = df.dtypes[df.dtypes == 'datetime64[ns]'].index.to_list()

In [ ]:

proc_df = extract_datetime_features(proc_df, 
                                    datetime_cols, 
                                    include_epoch=True, 
                                    datetime_attributes=datetime_attributes)

## Train Test Split

In [ ]:
# Random Validation Set
from sklearn.model_selection import train_test_split 

n_valid = 12000
train_df, valid_df = train_test_split(proc_df, test_size=n_valid, random_state=42)

In [ ]:
# Temporal Validation Set

def train_test_split_temporal(df, datetime_column, n_valid):
    idx_sort = np.argsort(df[datetime_column])
    idx_train, idx_test = idx_sort[:-n_valid], idx_sort[-n_valid:]
    return df.iloc[idx_train, :], df.iloc[idx_test, :]

n_valid = 12000
datetime_column = 'saledate' 
train_df, valid_df = train_test_split_temporal(proc_df, datetime_column, n_valid)
train_df.shape, valid_df.shape